In [ ]:
!unzip 'Tiny ImageNet dataset.zip' -d tiny-imagenet-another/

In [ ]:
!pip install torch torchvision

In [7]:
import torch
from torch import nn, optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to EfficientNet's expected input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
])

# Load the Tiny ImageNet validation dataset
val_data = datasets.ImageFolder(root='tiny-imagenet-another/tiny-imagenet-200/val/images', transform=transform)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

# Load the pretrained EfficientNet model
model = models.efficientnet_b0(pretrained=True)

# Modify the classifier to match the Tiny ImageNet (200 classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 200)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set model to evaluation mode
model.eval()

# Evaluate the model
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = (correct / total) * 100
print(f'Accuracy on Tiny ImageNet validation set: {accuracy:.2f}%')


FileNotFoundError: Couldn't find any class folder in tiny-imagenet-another/tiny-imagenet-200/val/images.

In [ ]:
!pip install efficientnet_pytorch pandas

In [19]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
from PIL import Image
import pandas as pd
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for testing
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the WordNet to Label Mapping
def load_wordnet_mappings(mapping_file):
    wordnet_to_label = {}
    with open(mapping_file, 'r') as f:
        for line in f:
            wordnet_id, label = line.strip().split('\t')
            wordnet_to_label[wordnet_id] = label
    return wordnet_to_label

# Custom Dataset class for Tiny ImageNet validation data
class TinyImageNetDataset(Dataset):
    def __init__(self, img_folder, annotations_file, transform=None):
        self.img_folder = img_folder
        self.annotations = pd.read_csv(annotations_file, sep='\t', header=None, names=['image', 'class', 'x1', 'y1', 'x2', 'y2'])
        self.transform = transform
        
        # Mapping class labels to integers (since the class ids in the annotations might be in string format)
        self.class_to_idx = {cls: idx for idx, cls in enumerate(sorted(self.annotations['class'].unique()))}
        
    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = self.annotations.iloc[idx, 0]
        img_path = os.path.join(self.img_folder, img_name)
        image = Image.open(img_path).convert('RGB')
        
        # Get the label from the annotations
        class_name = self.annotations.iloc[idx, 1]
        label = self.class_to_idx[class_name]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Load the validation dataset
img_folder = 'tiny-imagenet-another/tiny-imagenet-200/val/images'  # Update with the path to the 'val_images' folder
annotations_file = 'tiny-imagenet-another/tiny-imagenet-200/val/val_annotations.txt'  # Update with the path to the annotations file
wordnet_mapping_file = 'tiny-imagenet-another/tiny-imagenet-200/words.txt'  # Path to the WordNet to label mapping file

# Load WordNet mappings
wordnet_to_label = load_wordnet_mappings(wordnet_mapping_file)

# Initialize the dataset and DataLoader
test_dataset = TinyImageNetDataset(img_folder=img_folder, annotations_file=annotations_file, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

# Load pre-trained EfficientNet model
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=200)  # Tiny ImageNet has 200 classes
model = model.to(device)
model.eval()  # Set the model to evaluation mode

# Define loss function (optional, just for reporting the loss)
criterion = nn.CrossEntropyLoss()

# Evaluate the model
# Evaluate the model
def evaluate(model, test_loader, criterion, device, wordnet_to_label):
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

            # Map predicted indices to WordNet labels
            predicted_classes = [wordnet_to_label.get(test_loader.dataset.annotations.iloc[i, 1], 'Unknown') for i in predicted.cpu().numpy()]
            # print(f"Predicted classes: {predicted_classes}")

    accuracy = 100.0 * correct / total
    avg_loss = running_loss / len(test_loader)
    return avg_loss, accuracy


# Run evaluation
test_loss, test_accuracy = evaluate(model, test_loader, criterion, device, wordnet_to_label)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


Loaded pretrained weights for efficientnet-b0


Evaluating: 100%|██████████| 625/625 [01:06<00:00,  9.43it/s]

Test Loss: 5.3179, Test Accuracy: 0.55%


In [21]:
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

# class TinyImageNetDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = root_dir
#         self.transform = transform
#         self.annotations = self._load_annotations()

#     def _load_annotations(self):
#         annotations = []
#         for folder in os.listdir(self.root_dir):
#             folder_path = os.path.join(self.root_dir, folder)
#             if os.path.isdir(folder_path):
#                 image_folder = os.path.join(folder_path, 'images')
#                 boxes_file = os.path.join(folder_path, f'{folder}_boxes.txt')
                
#                 # The folder name represents the class_id
#                 class_id = folder
                
#                 if os.path.exists(boxes_file):
#                     with open(boxes_file, 'r') as f:
#                         lines = f.readlines()
#                     for line in lines:
#                         parts = line.split()
#                         if len(parts) == 5:  # Ensure there are exactly 5 elements
#                             img_name, xmin, ymin, xmax, ymax = parts
#                             img_path = os.path.join(image_folder, img_name)
#                             annotations.append((img_path, class_id, int(xmin), int(ymin), int(xmax), int(ymax)))  # store bbox
#         return annotations

#     def __len__(self):
#         return len(self.annotations)

#     def __getitem__(self, idx):
#         img_path, class_id, xmin, ymin, xmax, ymax = self.annotations[idx]
#         img = Image.open(img_path).convert('RGB')
        
#         if self.transform:
#             img = self.transform(img)
        
#         return img, class_id, xmin, ymin, xmax, ymax  # Return bbox as well if needed


import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

import os
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class TinyImageNetDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.class_to_idx = self._get_class_to_idx()  # Map class names to integers
        self.annotations = self._load_annotations()

    def _get_class_to_idx(self):
        """
        Maps folder names (class IDs) to integer labels.
        """
        classes = sorted(os.listdir(self.root_dir))
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        return class_to_idx

    def _load_annotations(self):
        annotations = []
        for folder in os.listdir(self.root_dir):
            folder_path = os.path.join(self.root_dir, folder)
            if os.path.isdir(folder_path):
                image_folder = os.path.join(folder_path, 'images')
                boxes_file = os.path.join(folder_path, f'{folder}_boxes.txt')
                
                # Get the class_id as an integer from the mapping
                class_id = self.class_to_idx[folder]
                
                if os.path.exists(boxes_file):
                    with open(boxes_file, 'r') as f:
                        lines = f.readlines()
                    for line in lines:
                        parts = line.split()
                        if len(parts) == 5:  # Ensure there are exactly 5 elements in the line
                            img_name, xmin, ymin, xmax, ymax = parts
                            img_path = os.path.join(image_folder, img_name)
                            annotations.append((img_path, class_id, int(xmin), int(ymin), int(xmax), int(ymax)))  # store bbox
        return annotations

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path, class_id, xmin, ymin, xmax, ymax = self.annotations[idx]
        img = Image.open(img_path).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
        
        # Return image and class_id only for classification purposes
        return img, class_id




# Define your transformations for data augmentation and preprocessing
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Initialize Dataset and DataLoader
train_dataset = TinyImageNetDataset(root_dir='/home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/train', transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)



fine tuning

In [22]:
from efficientnet_pytorch import EfficientNet

# Load pre-trained EfficientNet-B0 model
model = EfficientNet.from_pretrained('efficientnet-b0')

# Replace the final fully connected layer for 200 classes
model._fc = torch.nn.Linear(model._fc.in_features, 200)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Freeze all layers except the final layer
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last fully connected layer
for param in model._fc.parameters():
    param.requires_grad = True


Loaded pretrained weights for efficientnet-b0


In [24]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Fine-tuning loop
def fine_tune(model, train_loader, criterion, optimizer, device, epochs=5):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)  # Move images and labels to device

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Update running statistics
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        avg_loss = running_loss / len(train_loader)
        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Train the model
fine_tune(model, train_loader, criterion, optimizer, device, epochs=2)


Epoch 1/2: 100%|██████████| 6250/6250 [12:43<00:00,  8.18it/s]


Epoch 1/2 - Loss: 4.0796, Accuracy: 29.31%


Epoch 2/2: 100%|██████████| 6250/6250 [12:42<00:00,  8.20it/s]

Epoch 2/2 - Loss: 2.9645, Accuracy: 40.48%


In [25]:
# Save the fine-tuned model
torch.save(model.state_dict(), 'fine_tuned_model.pth')

In [65]:
class TinyImageNetValidationDataset(Dataset):
    def __init__(self, root_dir, annotations_file, class_to_idx, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.class_to_idx = class_to_idx
        self.annotations = self._load_annotations(annotations_file)

    def _load_annotations(self, annotations_file):
        annotations = []
        with open(annotations_file, 'r') as f:
            lines = f.readlines()
        for line in lines:
            parts = line.split()
            if len(parts) >= 2:
                img_name = parts[0]
                class_id = self.class_to_idx[parts[1]]  # Map WordNet ID to integer class ID
                img_path = os.path.join(self.root_dir, 'images', img_name)
                annotations.append((img_path, class_id))
        return annotations

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path, class_id = self.annotations[idx]
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img, class_id




# Create WordNet ID to integer class mapping
class_to_idx = {wnid: idx for idx, wnid in enumerate(sorted(os.listdir(train_dataset.root_dir)))}

val_dataset = TinyImageNetValidationDataset(
    root_dir='/home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/val',
    annotations_file='/home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/val/val_annotations.txt',
    class_to_idx=class_to_idx,
    transform=test_transform
)

val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [32]:
class TinyImageNetTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = sorted([os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith('.JPEG')])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img, img_path  # Return image and its path for identification


In [ ]:
# Define the test dataset and loader
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Load the fine-tuned model for testing
model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = torch.nn.Linear(model._fc.in_features, 200)  # Ensure the output layer is correct
model.load_state_dict(torch.load('fine_tuned_model.pth'))
model = model.to(device)
model.eval()

# Evaluate the model
def evaluate(model, test_loader, device):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = outputs.max(1)
            
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Test the model
evaluate(model, val_loader, device)


Loaded pretrained weights for efficientnet-b0


/tmp/ipykernel_601977/3332869965.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('fine_tuned_model.pth'))
Testing: 100%|██████████| 625

Test Accuracy: 54.27%


In [35]:
def inference(model, data_loader, device):
    results = []
    with torch.no_grad():
        for images, img_paths in tqdm(data_loader, desc="Testing"):
            images = images.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = outputs.max(1)

            for img_path, pred in zip(img_paths, predicted):
                results.append((img_path, pred.item()))

    return results


In [37]:
test_dataset = TinyImageNetTestDataset(root_dir='/home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/test/images', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

test_results = inference(model, test_loader, device)

# Print some results
for img_path, pred in test_results[:10]:
    print(f"Image: {img_path}, Predicted Class: {pred}")


Testing: 100%|██████████| 625/625 [01:07<00:00,  9.28it/s]

Image: /home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/test/images/test_0.JPEG, Predicted Class: 130
Image: /home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/test/images/test_1.JPEG, Predicted Class: 179
Image: /home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/test/images/test_10.JPEG, Predicted Class: 178
Image: /home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/test/images/test_100.JPEG, Predicted Class: 159
Image: /home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/test/images/test_1000.JPEG, Predicted Class: 26
Image: /home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/test/images/test_1001.JPEG, Predicted Class: 112
Image: /home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/test/images/test_1002.JPEG, Predicted Class: 113
Image: /home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/test/images/test_1003.JPEG, Predicted Class: 53
Image: /home/btp_9/EfficientNet/tiny-imagen

In [71]:
test_dataset = TinyImageNetTestDataset(root_dir='/home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/test/images', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [74]:
import torch
from efficientnet_pytorch import EfficientNet
from torchvision import transforms
from torch.utils.data import DataLoader
import os
from tqdm import tqdm

# Step 1: Load Pretrained and Fine-Tuned Model
model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = torch.nn.Linear(model._fc.in_features, 200)  # 200 output classes
model.load_state_dict(torch.load('fine_tuned_model.pth'))
model.eval()

# Step 2: Add QuantStub and DeQuantStub for Static Quantization
class QuantizedEfficientNet(torch.nn.Module):
    def __init__(self, model):
        super(QuantizedEfficientNet, self).__init__()
        self.quant = torch.quantization.QuantStub()
        self.model = model
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.model(x)
        x = self.dequant(x)
        return x

quantized_model = QuantizedEfficientNet(model)

# Step 3: Fuse Layers (No fusing is required here as EfficientNet already includes fused layers)
# For other models, you might need to fuse Conv2d + BatchNorm + ReLU, etc.

# Step 4: Prepare the Model for Static Quantization
quantized_model.eval()
quantized_model.qconfig = torch.quantization.get_default_qconfig('fbgemm')  # Use 'fbgemm' for x86 CPUs

# Prepare the model for static quantization
torch.quantization.prepare(quantized_model, inplace=True)

# Step 5: Calibrate the Model Using Representative Dataset
# Use a subset of your validation or training dataset for calibration
def calibrate(model, loader):
    with torch.no_grad():
        for images, _ in tqdm(loader, desc="Calibrating Model"):
            model(images)

# Example data loader for calibration
calibration_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # Adjust dataset as needed
calibrate(quantized_model, calibration_loader)

# Step 6: Convert to a Quantized Model
torch.quantization.convert(quantized_model, inplace=True)

# Step 7: Save and Compare Model Sizes
torch.save(quantized_model.state_dict(), 'quantized_model.pth')

original_size = os.path.getsize('fine_tuned_model.pth')
quantized_size = os.path.getsize('quantized_model.pth')

print(f"Original Model Size: {original_size / 1e6:.2f} MB")
print(f"Quantized Model Size: {quantized_size / 1e6:.2f} MB")



Loaded pretrained weights for efficientnet-b0


/tmp/ipykernel_601977/4120950662.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('fine_tuned_model.pth'))
Calibrating Model: 100%|█████

Original Model Size: 17.35 MB
Quantized Model Size: 16.62 MB


In [76]:
# Step 8: Evaluate the Static Quantized Model
def evaluate_static_quantized_model(model, test_loader, class_names=None):
    model.eval()
    model.to('cpu')
    predictions = []
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating Quantized Model"):
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

            # Calculate accuracy if labels are available
            if labels is not None:
                correct += (predicted.cpu() == labels).sum().item()
                total += labels.size(0)

    if total > 0:
        accuracy = 100 * correct / total
        print(f"Accuracy: {accuracy:.2f}%")
    else:
        print("No labels available for accuracy calculation.")

    return predictions

# Use the test loader to evaluate the quantized model
evaluate_static_quantized_model(quantized_model, test_loader)


Evaluating Quantized Model:   0%|          | 0/625 [00:00<?, ?it/s]


NotImplementedError: Could not run 'aten::_slow_conv2d_forward' with arguments from the 'QuantizedCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::_slow_conv2d_forward' is only available for these backends: [CPU, CUDA, Meta, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorch

### Static Quantization using torchvision

In [77]:
!pip install torchvision --upgrade


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [79]:
from torchvision.models import efficientnet_b0

# Load the pre-trained model
model = efficientnet_b0(pretrained=True)

# Replace the classifier for 200 classes
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 200)




/home/btp_9/EfficientNet/venv311/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/btp_9/EfficientNet/venv311/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /home/btp_9/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 49.4MB/s]


In [81]:
print(model)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [80]:
from torch.quantization import fuse_modules

# Fuse Conv2d + BatchNorm2d + ReLU layers
def fuse_efficientnet(model):
    for module_name, module in model.named_children():
        if isinstance(module, torch.nn.Sequential):
            fuse_modules(module, ['0', '1', '2'], inplace=True)  # Fuse Conv2d, BatchNorm2d, ReLU
    return model

model = fuse_efficientnet(model)


AssertionError: did not find fuser method for: (<class 'torchvision.ops.misc.Conv2dNormActivation'>, <class 'torch.nn.modules.container.Sequential'>, <class 'torch.nn.modules.container.Sequential'>) 

### Dynamic Quantization - quantizes weights only, not useful

In [ ]:
import torch
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader

# Step 1: Load the Fine-Tuned Model
model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = torch.nn.Linear(model._fc.in_features, 200)  # Ensure 200 output classes
model.load_state_dict(torch.load('fine_tuned_model.pth'))
model.eval()

# Step 2: Quantize the Model (Dynamic Quantization)
quantized_model = torch.quantization.quantize_dynamic(
    model,  # Model to quantize
    {torch.nn.Linear},  # Layers to quantize (e.g., Linear layers)
    dtype=torch.qint8  # Quantization type
)

# Print model size comparison
original_size = torch.save(model.state_dict(), 'temp.pth') or os.path.getsize('temp.pth')
quantized_size = torch.save(quantized_model.state_dict(), 'temp_quantized.pth') or os.path.getsize('temp_quantized.pth')
os.remove('temp.pth')
os.remove('temp_quantized.pth')

print(f"Original Model Size: {original_size / 1e6:.2f} MB")
print(f"Quantized Model Size: {quantized_size / 1e6:.2f} MB")

# Step 3: Test the Quantized Model
def evaluate(model, test_loader, device, class_names=None):
    model.to('cpu')  # Force the model to use CPU for quantized operations
    model.eval()
    predictions = []  # To store predictions for all images
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing Quantized Model"):
            images, _ = batch  # Ignore file paths, we don't need them for evaluation
            images = images.to('cpu')  # Move the images to CPU

            # Forward pass
            outputs = model(images)
            _, predicted = outputs.max(1)

            # If class names are provided, map the indices to class names
            if class_names:
                predicted_classes = [class_names[idx] for idx in predicted]
                predictions.extend(predicted_classes)
            else:
                predictions.extend(predicted.cpu().numpy())  # If no class names, just return indices

    # Display some of the predictions
    print("Predictions for the first 10 images:")
    for i, prediction in enumerate(predictions[:10]):
        print(f"Image {i+1}: Predicted Class = {prediction}")

    return predictions






# Assuming test_loader is already defined for your test dataset

#######
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#  for the above line
# The error you're encountering suggests that the quantized model is trying to run on a CUDA device, but the operation quantized::linear_dynamic doesn't support the CUDA backend. 
# This is a known limitation with PyTorch quantization: certain operations, specifically linear_dynamic for quantized models, may not have support for execution on GPUs (CUDA). 
# This operation is typically supported only on CPU.
######
evaluate(quantized_model, test_loader, device)


Loaded pretrained weights for efficientnet-b0


/tmp/ipykernel_601977/4011583075.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('fine_tuned_model.pth'))


Original Model Size: 17.32 MB
Quantized Model Size: 16.58 MB


Testing Quantized Model: 100%|██████████| 625/625 [02:07<00:00,  4.90it/s]

Predictions for the first 10 images:
Image 1: Predicted Class = 130
Image 2: Predicted Class = 190
Image 3: Predicted Class = 178
Image 4: Predicted Class = 159
Image 5: Predicted Class = 26
Image 6: Predicted Class = 112
Image 7: Predicted Class = 113
Image 8: Predicted Class = 53
Image 9: Predicted Class = 105
Image 10: Predicted Class = 78


[np.int64(130),
 np.int64(190),
 np.int64(178),
 np.int64(159),
 np.int64(26),
 np.int64(112),
 np.int64(113),
 np.int64(53),
 np.int64(105),
 np.int64(78),
 np.int64(93),
 np.int64(114),
 np.int64(36),
 np.int64(8),
 np.int64(54),
 np.int64(37),
 np.int64(41),
 np.int64(118),
 np.int64(66),
 np.int64(153),
 np.int64(83),
 np.int64(52),
 np.int64(41),
 np.int64(170),
 np.int64(191),
 np.int64(114),
 np.int64(189),
 np.int64(2),
 np.int64(30),
 np.int64(98),
 np.int64(74),
 np.int64(122),
 np.int64(184),
 np.int64(6),
 np.int64(160),
 np.int64(156),
 np.int64(119),
 np.int64(27),
 np.int64(191),
 np.int64(57),
 np.int64(105),
 np.int64(103),
 np.int64(4),
 np.int64(78),
 np.int64(4),
 np.int64(20),
 np.int64(76),
 np.int64(94),
 np.int64(72),
 np.int64(1),
 np.int64(161),
 np.int64(71),
 np.int64(34),
 np.int64(193),
 np.int64(148),
 np.int64(0),
 np.int64(83),
 np.int64(43),
 np.int64(150),
 np.int64(155),
 np.int64(58),
 np.int64(187),
 np.int64(23),
 np.int64(44),
 np.int64(150),
 np

In [68]:
val_dataset = datasets.ImageFolder(root="/home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/val_restructured", transform=transform)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [69]:
def test(model, dataloader):
    model.to('cpu')
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total * 100
    return accuracy

# Evaluate quantized model on Tiny ImageNet validation dataset
accuracy = test(model, val_loader)
print(f'Accuracy of the quantized EfficientNet on Tiny ImageNet: {accuracy:.2f}%')



Accuracy of the quantized EfficientNet on Tiny ImageNet: 46.10%


In [60]:
def load_class_names(file_path):
    class_ids = []  # List of class IDs (e.g., n00001740)
    class_names = {}  # Mapping from class ID to class name
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split("\t")
            class_id = parts[0]
            class_name = parts[1]
            class_ids.append(class_id)  # Store the class ID
            class_names[class_id] = class_name  # Map class ID to class name
    return class_ids, class_names

def evaluate(model, test_loader, device, class_ids, class_names):
    model.to('cpu')  # Force the model to use CPU for quantized operations
    model.eval()
    predictions = []  # To store predictions for all images
    predictions_class_ids = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing Quantized Model"):
            images, _ = batch  # Ignore file paths, we don't need them for evaluation
            images = images.to('cpu')  # Move the images to CPU

            # Forward pass
            outputs = model(images)
            _, predicted = outputs.max(1)

            # Map predicted class indices to class IDs
            predicted_class_ids = [class_ids[idx] for idx in predicted.cpu().numpy()]
            predicted_classes = [class_names[class_id] for class_id in predicted_class_ids]
            predictions.extend(predicted_classes)
            predictions_class_ids.extend(predicted.cpu().numpy())

    # Display some of the predictions
    print("Predictions for the first 10 images:")
    for i, prediction in enumerate(predictions[:10]):
        print(f"Image {i+1}: Predicted Class = {prediction}")

    for i, prediction in enumerate(predictions_class_ids[:10]):
        print(f"Image {i+1}: Predicted Class = {prediction}")

    return predictions


In [61]:
class_ids, class_names = load_class_names('/home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/words.txt')  # Provide the correct path
predictions = evaluate(quantized_model, test_loader, device, class_ids, class_names)


Testing Quantized Model: 100%|██████████| 625/625 [02:07<00:00,  4.91it/s]

Predictions for the first 10 images:
Image 1: Predicted Class = penetration
Image 2: Predicted Class = male orgasm
Image 3: Predicted Class = exodus, hegira, hejira
Image 4: Predicted Class = retreat
Image 5: Predicted Class = nutrient
Image 6: Predicted Class = arrival, reaching
Image 7: Predicted Class = arrival
Image 8: Predicted Class = abdominoplasty, tummy tuck
Image 9: Predicted Class = tour de force
Image 10: Predicted Class = reciprocity
Image 1: Predicted Class = 130
Image 2: Predicted Class = 190
Image 3: Predicted Class = 178
Image 4: Predicted Class = 159
Image 5: Predicted Class = 26
Image 6: Predicted Class = 112
Image 7: Predicted Class = 113
Image 8: Predicted Class = 53
Image 9: Predicted Class = 105
Image 10: Predicted Class = 78


In [ ]:
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import numpy as np

# Define a custom dataset
class CustomTestDataset(Dataset):
    def __init__(self, image_folder, annotations_file, class_names):
        self.image_folder = image_folder
        self.class_names = class_names
        
        # Read the annotations
        with open(annotations_file, 'r') as file:
            self.annotations = file.readlines()

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Get the annotation for this image
        line = self.annotations[idx].strip().split()
        img_name = line[0]  # e.g., 'val_0.JPEG'
        class_id = line[1]  # e.g., 'n03444034'
        bbox = list(map(int, line[2:]))  # Bounding box: [x1, y1, x2, y2]
        
        # Load the image
        img_path = os.path.join(self.image_folder, img_name)
        image = Image.open(img_path).convert('RGB')
        
        # Convert to tensor and normalize (if needed)
        image = torch.tensor(np.array(image)).float()  # Or apply any necessary transforms
        
        # Map class_id to class name
        class_name = self.class_names.get(class_id, "Unknown")
        
        return image, class_name, bbox

# Load class names (from your words.txt or any other mapping)
def load_class_names(file_path):
    class_names = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_names[parts[0]] = parts[1]  # Map class ID to name
    return class_names

# Example usage
image_folder = '/home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/val/images'  # Replace with your actual folder path
annotations_file = '/home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/val/val_annotations.txt'  # Replace with the actual file path
class_names = load_class_names('/home/btp_9/EfficientNet/tiny-imagenet-another/tiny-imagenet-200/words.txt')  # Provide the correct path to words.txt

# Create the dataset
test_dataset = CustomTestDataset(image_folder, annotations_file, class_names)

# Create DataLoader
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Iterate over the test loader
for images, class_names, bbox in test_loader:
    print(f"Image: {images}, Class: {class_names}, Bounding Box: {bbox}")


In [63]:
import torch
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

# Define a custom dataset for testing (like before)
class CustomTestDataset(Dataset):
    def __init__(self, image_folder, annotations_file, class_names):
        self.image_folder = image_folder
        self.class_names = class_names
        
        with open(annotations_file, 'r') as file:
            self.annotations = file.readlines()

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        line = self.annotations[idx].strip().split()
        img_name = line[0]
        class_id = line[1]
        bbox = list(map(int, line[2:]))
        
        img_path = os.path.join(self.image_folder, img_name)
        image = Image.open(img_path).convert('RGB')
        
        image = torch.tensor(np.array(image)).float()  # Normalize or apply transforms
        
        # Convert class_id to class name
        class_name = self.class_names.get(class_id, "Unknown")
        
        return image, class_name, class_id  # Also return the true class_id for comparison

# Assuming you have the model and class_names loaded
def calculate_accuracy(model, test_loader, device):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    
    with torch.no_grad():
        for images, true_class_names, true_class_ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the class with max probability

            # Compare predicted class with true class
            correct_predictions += (predicted == true_class_ids.to(device)).sum().item()
            total_predictions += true_class_ids.size(0)
    
    accuracy = 100 * correct_predictions / total_predictions
    return accuracy

# Create DataLoader for test set
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Calculate accuracy
accuracy = calculate_accuracy(model, test_loader, device)
print(f'Accuracy: {accuracy:.2f}%')


RuntimeError: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 64, 65, 4] to have 3 channels, but got 64 channels instead